In [112]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.


In [249]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [250]:
import pandas as pd
df_house = pd.read_csv('City_Zhvi_AllHomes.csv')
df_house.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04
0,6181,New York,NY,New York-Newark-Jersey City,Queens County,1,169300.0,169800.0,170300.0,170600.0,...,670100,670200,672900,677100,680100,680400,679300,679400,678900,677000
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,2,157700.0,157600.0,157500.0,157300.0,...,676900,678900,682200,686100,688800,690300,691500,691100,688700,686100
2,39051,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,3,86500.0,86600.0,86600.0,86700.0,...,180000,181100,181800,182300,183200,184500,185800,187200,187200,186000
3,17426,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,114000.0,115100.0,116100.0,116600.0,...,227200,227100,227300,227600,227800,228200,228500,229400,230000,229400
4,6915,San Antonio,TX,San Antonio-New Braunfels,Bexar County,5,86600.0,86400.0,86300.0,86300.0,...,169200,170300,171500,172700,173900,175000,175900,176500,176600,176100


In [251]:
df_gdp_year = pd.read_csv('gdplev1.csv')
df_gdp_quar = pd.read_csv('gdplev2.csv')

In [252]:
df_gdp_quar.head()

,Quarter,GDP in billions of current dollars,GDP in billions of chained 2009 dollars
0,1947q1,243.1,"1,934.5"
1,1947q2,246.3,"1,932.3"
2,1947q3,250.1,"1,930.3"
3,1947q4,260.3,"1,960.7"
4,1948q1,266.2,"1,989.5"


In [253]:
df = pd.read_table('university_towns.txt',header = None)

C:\Users\Erick\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [254]:
df.head()

,0
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]


In [255]:
town = str(df.iloc[1].values)[2:str(df.iloc[1].values).find('(') - 1]
state = str(df.iloc[0].values)[2:-8]

In [256]:
state
town

'Auburn'

In [257]:
town.find('(')

-1

In [375]:
def get_list_of_university_towns():
    df = pd.read_table('university_towns.txt',header = None)
    state = str(df.iloc[0].values)[2:-8]
    list1 = []
    #df1 = pd.DataFrame(list1,columns = ['State','RegionName'])
    for i in range(1,len(df)):
        if '[edit]' not in str(df.iloc[i].values):
            list1.append([state, str(df.iloc[i].values)[2:str(df.iloc[i].values).find('(') - 1]])
        else:
            state = str(df.iloc[i].values)[2:-8]
    df1 = pd.DataFrame(list1)
    df1.rename(columns={0:'State', 1:'RegionName'}, inplace=True)
    df1.set_index(['State','RegionName'], inplace = True)
    return df1

In [376]:
df1.head()

Empty DataFrame
Columns: []
Index: [(Alabama, Auburn), (Alabama, Florence), (Alabama, Jacksonville), (Alabama, Livingston), (Alabama, Montevallo)]

In [277]:
n = df_gdp_quar.loc[df_gdp_quar['Quarter']=='2000q1'].index
n

Int64Index([212], dtype='int64')

In [278]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    #your code here
    for i in range(212, len(df_gdp_quar)):
        if (df_gdp_quar['GDP in billions of chained 2009 dollars'][i] > df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 1]) and (df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 1] > df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 2]) and (df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 2] < df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 3]) and (df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 3] < df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 4]):
            record = i
            break
    return str(df_gdp_quar['Quarter'][i]) 

In [279]:
rec_start = get_recession_start()
rec_start

'2008q4'

In [280]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    #your code here
    for i in range(212, len(df_gdp_quar)):
        if (df_gdp_quar['GDP in billions of chained 2009 dollars'][i] > df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 1]) and (df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 1] > df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 2]) and (df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 2] < df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 3]) and (df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 3] < df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 4]):
            record = i
            break
    return str(df_gdp_quar['Quarter'][i + 4]) 

In [281]:
rec_end = get_recession_end()
rec_end

'2009q4'

In [282]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    #your code here
    for i in range(212, len(df_gdp_quar)):
        if (df_gdp_quar['GDP in billions of chained 2009 dollars'][i] > df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 1]) and (df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 1] > df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 2]) and (df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 2] < df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 3]) and (df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 3] < df_gdp_quar['GDP in billions of chained 2009 dollars'][i + 4]):
            record = i
            break
    return str(df_gdp_quar['Quarter'][i + 2]) 

In [283]:
rec_bottom = get_recession_bottom()
rec_bottom

'2009q2'

In [284]:
def new_col_names():
    #generating the new coloumns names 
    years = range(2000,2017)
    quars = ['q1','q2','q3','q4']
    quar_years = []
    for i in years:
        for x in quars:
            quar_years.append((str(i)+x))
    return quar_years[:67]

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    #your code here
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    df_house = pd.read_csv('City_Zhvi_AllHomes.csv')
    cols = [0]
    #df_house.drop(df.columns[0],axis=1,inplace=True) #drop regionid

    df_house.replace({'State':states}, inplace = True)
    df_house.set_index(['State','RegionName'], inplace = True)
    a = new_col_names()
    colnum = 50
    for x in a:
        df_house[x] = (df_house.iloc[:, colnum] + df_house.iloc[:, colnum + 1] + df_house.iloc[:, colnum + 2]) / 3
        colnum += 1
    df_house.drop(df_house.columns[0:281],axis=1,inplace=True) #drops additional columns not related - including coverage from 1996-2019
    return df_house

In [285]:
a = convert_housing_data_to_quarters()

In [286]:
a.head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,223266.666667,226066.666667,228866.666667,231500.000000,233933.333333,236166.666667,238233.333333,240233.333333,242266.666667,244366.666667,...,422333.333333,427766.666667,433000.000000,438333.333333,443966.666667,449866.666667,455966.666667,462233.333333,468666.666667,475200.000000
California,Los Angeles,210433.333333,212666.666667,214900.000000,217100.000000,219300.000000,221466.666667,223566.666667,225633.333333,227766.666667,230000.000000,...,498100.000000,504566.666667,510966.666667,517600.000000,524600.000000,532133.333333,540166.666667,548533.333333,556933.333333,565100.000000
Texas,Houston,106366.666667,106366.666667,106300.000000,106233.333333,106200.000000,106233.333333,106300.000000,106400.000000,106500.000000,106566.666667,...,125666.666667,126166.666667,126600.000000,126933.333333,127166.666667,127300.000000,127333.333333,127266.666667,127066.666667,126833.333333
Illinois,Chicago,143966.666667,145666.666667,147433.333333,149233.333333,151100.000000,153033.333333,154966.666667,156866.666667,158700.000000,160466.666667,...,231933.333333,233600.000000,235266.666667,237066.666667,239066.666667,241200.000000,243466.666667,245800.000000,248166.666667,250500.000000
Texas,San Antonio,92100.000000,91966.666667,91800.000000,91566.666667,91333.333333,91133.333333,91000.000000,90900.000000,90800.000000,90666.666667,...,104933.333333,105466.666667,106000.000000,106466.666667,106866.666667,107200.000000,107466.666667,107666.666667,107833.333333,108000.000000


In [372]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
  #your code here
    
    uni_towns = get_list_of_university_towns()
    begin = get_recession_start()
    ending = get_recession_end()
    lowest = get_recession_bottom()
    all_towns = convert_housing_data_to_quarters()
    
    prices_begin = house[begin]
    prices_end = house[ending]
    ratio_all = prices_begin.divide(prices_end)
    
    all_towns_index = all_towns.index
    uni_towns_index = uni_towns.index 
    not_uni_index = all_towns_index.difference(uni_towns_index)
    
    ratio_uni = ratio_all.loc[uni_towns_index].dropna()
    ratio_not_uni = ratio_all.loc[not_uni_index].dropna()
        
    statistic, p_value = tuple(ttest_ind(ratio_uni, ratio_not_uni))
    
    outcome = statistic < 0
    
    different = p_value < 0.01
    
    better = ["non-university town", "university town"]
    
    return (different, p_value, better[outcome])

In [377]:
t = run_ttest()
print(t)

C:\Users\Erick\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


(False, 0.2225003313969281, 'non-university town')


C:\Users\Erick\Anaconda2\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\Erick\Anaconda2\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
